# Assignment-4 COMP-5630 Jacob Murrah
## README
This notebook implements...

## Dependencies
- **Python 3.x**
- **pandas**
- **numpy**
- **matplotlib**
- **sklearn**

*Note: If you are running this notebook in Google Colab, all the required packages are pre-installed.*

## Instructions
1. **Run All Cells:** Please click on \"Runtime\" > \"Run all\" to execute the entire notebook sequentially.
2. **Review the Outputs:** The notebook is organized into several sections. Ensure that all cells run without errors.


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Part 1. Naive Bayes
See the attached housing data (Asssignment4_Data.xlsx). Each tab in the Excel file contains training and test splits. Your goal is to construct a Naïve Bayes classifier for this data.

In [4]:
train_data = pd.read_excel("Asssignment4_Data.xlsx", sheet_name="Train")
test_data = pd.read_excel("Asssignment4_Data.xlsx", sheet_name="Test")

## Part 1. (1) Compute and show the conditional probability distribution for each feature.<br>
Note: You are expected to do this part of the question by hand.<br>
**Answer:**<br>
Using Naive Bayes we will predict the price of a house given it's other features. The first step to creating a conditional probability is seperating the housing prices into groups. Since housing prices are continuous I will need to use thresholds to split the housing prices into groups. I choose to split the data into two equal groups (Low and High) with the seperator being the median.<br><br>
Median House Price = (5.6039 + 5.8282) / 2 = **5.7161**<br>
Low Price House Ids: 1, 2, 3, 4, 5, 6, 8, 12, 15, 16<br>
High Price House Ids: 7, 9, 10, 11, 13, 14, 17, 18, 19, 20<br>

| Price Category | Prior Probability |
| -------------- | ----------------- |
| Low Price      | 10/20 = 0.5       |
| High Price     | 10/20 = 0.5       |

Now I will calculate the conditional probabilities for each feature. For discrete features I will use tables and for continuous features I will use assume normality and calculate the mean and variance. Note that I will be using Add-One Smoothing for ALL of my calculations.
$$P(x_i | y) = \frac{count(x_i, y) + 1}{count(y) + |V|}$$
$$\bar{x} = \frac{1}{n} \sum_{i=1}^{n} x_i$$
$$s^2 = \frac{1}{n - 1} \sum_{i=1}^{n} \bigl(x_i - \bar{x}\bigr)^2$$

**Bathrooms Feature (Discrete)**:

| # Bathrooms (LOW PRICE) | Count | P(# Bath = X \| LOW)  |
|-------------------------|-------|-----------------------|
| 1                       | 10    | (10+1)/(10+3) ≈ 0.846 |
| 1.5                     | 0     | (0+1)/(10+3) ≈ 0.077  |
| 2.5                     | 0     | (0+1)/(10+3) ≈ 0.077  |

| # Bathrooms (HIGH PRICE) | Count | P(# Bath = X \| HIGH) |
|--------------------------|-------|-----------------------|
| 1                        | 5     | (5+1)/(10+3) ≈ 0.462  |
| 1.5                      | 3     | (3+1)/(10+3) ≈ 0.308  |
| 2.5                      | 2     | (2+1)/(10+3) ≈ 0.231  |

<br>**Garages Feature (Discrete)**:

| # Garages (LOW PRICE) | Count | P(# Garages = X \| LOW) |
|-----------------------|-------|-------------------------|
| 0                     | 3     | (3+1)/(10+4) ≈ 0.286    |
| 1                     | 6     | (6+1)/(10+4) ≈ 0.500    |
| 1.5                   | 0     | (0+1)/(10+4) ≈ 0.071    |
| 2                     | 1     | (1+1)/(10+4) ≈ 0.143    |

| # Garages (HIGH PRICE) | Count | P(# Garages = X \| HIGH) |
|------------------------|-------|--------------------------|
| 0                      | 0     | (0+1)/(10+4) ≈ 0.071     |
| 1                      | 3     | (3+1)/(10+4) ≈ 0.286     |
| 1.5                    | 2     | (2+1)/(10+4) ≈ 0.214     |
| 2                      | 5     | (5+1)/(10+4) ≈ 0.429     |

<br>**Rooms Feature (Discrete)**:

| # Rooms (LOW PRICE)     | Count | P(# Rooms = X \| LOW)  |
|-------------------------|-------|------------------------|
| 5                       | 1     | (1+1)/(10+6) ≈ 0.125   |
| 6                       | 7     | (7+1)/(10+6) ≈ 0.500   |
| 7                       | 2     | (2+1)/(10+6) ≈ 0.1875  |
| 8                       | 0     | (0+1)/(10+6) ≈ 0.0625  |
| 9                       | 0     | (0+1)/(10+6) ≈ 0.0625  |
| 10                      | 0     | (0+1)/(10+6) ≈ 0.0625  |

| # Rooms (HIGH PRICE)    | Count | P(# Rooms = X \| HIGH) |
|-------------------------|-------|------------------------|
| 5                       | 1     | (1+1)/(10+6) ≈ 0.125   |
| 6                       | 3     | (3+1)/(10+6) ≈ 0.250   |
| 7                       | 3     | (3+1)/(10+6) ≈ 0.250   |
| 8                       | 1     | (1+1)/(10+6) ≈ 0.125   |
| 9                       | 1     | (1+1)/(10+6) ≈ 0.125   |
| 10                      | 1     | (1+1)/(10+6) ≈ 0.125   |

<br>**Bedrooms Feature (Discrete)**:

| # Bedrooms (LOW PRICE)  | Count | P(# Bedrooms = X \| LOW) |
|-------------------------|-------|--------------------------|
| 2                       | 1     | (1+1)/(10+4) ≈ 0.143     |
| 3                       | 7     | (7+1)/(10+4) ≈ 0.571     |
| 4                       | 2     | (2+1)/(10+4) ≈ 0.214     |
| 5                       | 0     | (0+1)/(10+4) ≈ 0.0714    |

| # Bedrooms (HIGH PRICE) | Count | P(# Bedrooms = X \| HIGH) |
|-------------------------|-------|---------------------------|
| 2                       | 1     | (1+1)/(10+4) ≈ 0.143      |
| 3                       | 6     | (6+1)/(10+4) ≈ 0.500      |
| 4                       | 1     | (1+1)/(10+4) ≈ 0.143      |
| 5                       | 2     | (2+1)/(10+4) ≈ 0.214      |

<br>**Construction Type Feature (Discrete)**:

| Construction Type (LOW PRICE) | Count | P(Construction Type = X \| LOW)  |
|-------------------------|-------|-----------------------------|
| Apartment               | 4     | (4+1)/(10+3) ≈ 0.385        |
| Condo                   | 2     | (2+1)/(10+3) ≈ 0.231        |
| House                   | 4     | (4+1)/(10+3) ≈ 0.385        |

| Construction Type (HIGH PRICE) | Count | P(Construction Type = X \| HIGH)  |
|-------------------------|-------|-----------------------------|
| Apartment               | 3     | (3+1)/(10+3) ≈ 0.308 |
| Condo                   | 4     | (4+1)/(10+3) ≈ 0.385 |
| House                   | 3     | (3+1)/(10+3) ≈ 0.308 |

<br>**Land Area Feature (Continuous)**:

| Price Type | Sample Mean ($\bar{x}$)  | Sample Variance ($s^2$)   |
|------------|-------|-----------------------|
| Low Price  | $\frac{51.2663}{10}\approx 5.1267$ | $\frac{60.1283}{10}\approx 6.0128$ |
| High Price | $\frac{74.025}{10}\approx 7.4025$  | $\frac{53.2068}{10}\approx 5.3207$ |

<br>**Living Area Feature (Continuous)**:

| Price Type | Sample Mean ($\bar{x}$)  | Sample Variance ($s^2$)   |
|------------|-------|-----------------------|
| Low Price  | $\frac{12.588}{10}\approx 1.2588$ | $\frac{0.5406}{10}\approx 0.0541$ |
| High Price | $\frac{17.009}{10}\approx 1.7009$  | $\frac{6.8197}{10}\approx 0.682$ |

<br>**Age of Home Feature (Continuous)**:

| Price Type | Sample Mean ($\bar{x}$)  | Sample Variance ($s^2$)   |
|------------|-------|-----------------------|
| Low Price  | $\frac{436}{10}\approx 43.6$ | $\frac{1198.2222}{10}\approx 119.8222$ |
| High Price | $\frac{313}{10}\approx 31.3$  | $\frac{1682.3333}{10}\approx 168.2333$ |